In [ ]:
import networkx as nx
from matplotlib import pyplot, patches
from scipy import io
import numpy as np
from collections import defaultdict
import louvain
import pickle
from tqdm import tqdm
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt 

In [ ]:
# The MCMC Algorithm orgnized like
# 1. Using a random matrix as the beginning
# 2. give a random theta (alpha, zeta) and make it slight move theta' (alpha', zeta')
# 3. use theta' (alpha', zeta') generate a network with same nodes as the sample
# 4. calculate the sample's S1,S2,S3
# 5. Compare sample S1,S2,S3 with from the emprirical data
# repeat until a close area of theta' (alpha', zeta') which accept, calculate the std, mean

In [ ]:
def Pron_Mat_theta(Mat,alpha,zeta):
    
    N=len(Mat)
    incr_m=Mat
    decay_m=[[0] * N for row in range(N)]
    # degree for each mode 
    Degree=np.sum(Mat, axis=0)
   

    for i in range(N):
        Value={}
        Decline={}
        for j in range(i+1,N):  # the node i with j
            if Mat[i][j]==0:  # no link
                Value[j]=Degree[j] # degree is the value     
                    
            if Mat[i][j]==1:  # with link
                Decline[j]=Degree[j] # degree is the value
                
    #    print(Value)
        max_keys=0
        max_value=0
        if len(Value) >0:
            max_value = max(Value.values())  # maximum value
            max_keys = [k for k, v in Value.items() if v == max_value]    # maximun keys    
    #    print(max_keys)
        # count the number of maxium keys for divider
        if max_keys != 0:
            for k in max_keys:
                incr_m[i][k]=  alpha/len(max_keys)
                incr_m[k][i]=  alpha/len(max_keys)
          
                
        min_keys=0
        min_value=0
        if len(Decline) >0:
            min_value = min(Decline.values())  # min value
            min_keys = [k for k, v in Decline.items() if v == min_value]    # min keys    
            
        if min_keys != 0:
            for k in min_keys:
                decay_m[i][k]=  (1-alpha)/len(min_keys)
                decay_m[k][i]= (1- alpha)/len(min_keys)         
                            
        
    return  incr_m, decay_m

In [ ]:
def CreatMat(incr_m,decay_m):
    B=np.random.rand(*np.array(incr_m).shape)
    B_symm = (B + B.T)/2
    adjacency_add = B_symm  <= np.array(incr_m) # adjacency[ii, jj] is True with probability P[ii, jj]
    adjacency_de = B_symm  <= np.array(decay_m) # adjacency[ii, jj] is True with probability P[ii, jj]
    
    New_Mat=adjacency_add*1-adjacency_de*1
       
    
 #   print(New_Mat)
    return New_Mat.tolist()
    

In [ ]:
def Make_stable(Mat,n,alpha):
    for i in tqdm(range(n)):
        incr_m,decay_m=Prob_links(Mat,alpha)
        Mat=CreatMat(incr_m,decay_m)
            
    return Mat

In [ ]:
def Make_Ori_Mat(N):
    # generate the original matrix
    
    MATRIX=np.random.rand(N,N)
    np.fill_diagonal(MATRIX, 0)
    bollmat = 0.5  <= MATRIX
    nummat=1*bollmat
    randmat=np.minimum( nummat, nummat.transpose() )
    
    return randmat.tolist()

In [ ]:
def draw_adjacency_matrix(G, node_order=None, partitions=[], colors=[]):
    """
    - G is a netorkx graph
    - node_order (optional) is a list of nodes, where each node in G
          appears exactly once
    - partitions is a list of node lists, where each node in G appears
          in exactly one node list
    - colors is a list of strings indicating what color each
          partition should be
    If partitions is specified, the same number of colors needs to be
    specified.
    """
    adjacency_matrix = nx.to_numpy_matrix(G, dtype=np.bool, nodelist=node_order)

    #Plot adjacency matrix in toned-down black and white
    fig = pyplot.figure(figsize=(5, 5)) # in inches
    pyplot.imshow(adjacency_matrix,
                  cmap="Greys",
                  interpolation="none")
    
    # The rest is just if you have sorted nodes by a partition and want to
    # highlight the module boundaries
    assert len(partitions) == len(colors)
    ax = pyplot.gca()
    for partition, color in zip(partitions, colors):
        current_idx = 0
        for module in partition:
            ax.add_patch(patches.Rectangle((current_idx, current_idx),
                                          len(module), # Width
                                          len(module), # Height
                                          facecolor="none",
                                          edgecolor=color,
                                          linewidth="1"))
            current_idx += len(module)